In [1]:
import psycopg2, sys, pytz, re
from datetime import datetime
import pandas as pd

In [2]:
def updateT1Datasets():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")
        cur.execute("""SELECT DISTINCT samplingfeaturecode FROM team1_results_methods""")
        frows = cur.fetchall()
        
        for frow in frows:
            dsamplingfeaturecode = frow[0]

            cur.execute("""SELECT DISTINCT datasetcode, samplingfeaturename, resultdatetime
                           FROM team1_results_methods 
                           WHERE samplingfeaturecode = %(vsamplingfeaturecode)s""",
                        {'vsamplingfeaturecode': dsamplingfeaturecode})
            try:
                drow = cur.fetchall()
                ddatasetcode = drow[0][0]
                
                samplingfeaturename = drow[0][1].split(',')
                if len(samplingfeaturename) > 1:
                    splitnum = re.split('-1| - 1|-2| - 2', samplingfeaturename[1])
                    
                    samplingfeaturename = splitnum[0].strip().capitalize()+' '+samplingfeaturename[0].title()
                else:
                    samplingfeaturename = samplingfeaturename[0].split('CITY')
                    if len(samplingfeaturename) > 1:
                        splitnum = re.split('-1| - 1|-2| - 2|-3', samplingfeaturename[1])
                        samplingfeaturename = 'City '+splitnum[0].strip()+' '+samplingfeaturename[0].title()
                    else:  
                        samplingfeaturename = samplingfeaturename[0].capitalize()
                    
                resultdatetime = drow[0][2]
                resultdatetime = str(resultdatetime.strftime("%B"))+' '+str(resultdatetime.day)+', '+str(resultdatetime.year)

            except:
                sys.exit('Error on datasetcode')
                
            dlab = ddatasetcode.split('_')[1]
            
            if dlab == 'Knappe':
                try:
                    datasettitle = dsamplingfeaturecode[:-9]+': Stream and well measurements from '+samplingfeaturename+' site on '+resultdatetime
                    datasetabstract = 'Team 1 stream and well measurements from Dr. Knappe Lab at NCSU at station '+dsamplingfeaturecode[:-9]
                except:
                    sys.exit('Error on dsamplingfeaturecode')
            elif dlab == 'Ferguson':
                try:
                    datasettitle = dsamplingfeaturecode[:-9]+': Stream and well measurements from '+samplingfeaturename+' site on '+resultdatetime
                    datasetabstract = 'Team 1 stream and well measurements from Dr. Ferguson Lab at Duke University at station '+dsamplingfeaturecode[:-9]
                except:
                    sys.exit('Error on dsamplingfeaturecode')
            
            cur.execute("""UPDATE datasets
                               SET datasettitle = %(vdatasettitle)s, 
                                   datasetabstract = %(vdatasetabstract)s
                           WHERE datasetcode = %(vdatasetcode)s""",
                        {'vdatasetcode':ddatasetcode,'vdatasettitle':datasettitle,
                         'vdatasetabstract':datasetabstract})

            conn.commit()

        cur.execute("""ANALYZE datasets""")
        
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close()


In [3]:
updateT1Datasets()